In [17]:
import nltk
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import string
import text2emotion as te
import warnings
#import eda
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from textblob import TextBlob

In [12]:
df_train=pd.read_csv('FakeNews-Dataset//train.csv//train.csv')
df_train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [13]:
df_train.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
df_train=df_train.dropna()
df_train=df_train.reset_index()
df_train.drop(['index','id'],axis=1,inplace=True)
df_train.isna().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [15]:
#PREPROCESSING USING REGULAR EXPRESSION
def clean_texts(text):
    text = re.sub(r"that’s","that is",text)
    text = re.sub(r"there’s","there is",text)
    text = re.sub(r"what’s","what is",text)
    text = re.sub(r"where’s","where is",text)
    text = re.sub(r"it’s","it is",text)
    text = re.sub(r"who’s","who is",text)
    text = re.sub(r"i’m","i am",text)
    text = re.sub(r"she’s","she is",text)
    text = re.sub(r"he’s","he is",text)
    text = re.sub(r"they’re","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain’t","am not",text)
    text = re.sub(r"wouldn’t","would not",text)
    text = re.sub(r"shouldn’t","should not",text)
    text = re.sub(r"can’t","can not",text)
    text = re.sub(r"couldn’t","could not",text)
    text = re.sub(r"won’t","will not",text)
    text = re.sub(r"didn’t","did not",text)
    text = re.sub(r"doesn’t","does not",text)
    # remove URL
    text = re.sub(r"http\S+", " ", text)
#     # Remove usernames
    text = re.sub(r"@[^\s]+[\s]?",' ',text)
#     # remove special characters 
    text = re.sub('[^ a-zA-Z0-9]', ' ', text)
#     # remove Numbers
    text = re.sub('[0-9]', '', text)
    text = re.sub(' +', ' ',text)
    text = text.encode('ascii', 'ignore').decode()
    text = re.sub(r'@\S+', ' ', text)
    text = re.sub(r'#\S+', ' ', text)
    text = re.sub(r'\'\w+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)

    return text

In [16]:
df_train['text'] = df_train['text'].apply(clean_texts)
df_train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide We Didn t Even See Comey s Lett...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,Why the Truth Might Get You Fired October The ...,1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos Civilians Killed In Single US Airstrike...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print An Iranian woman has been sentenced to s...,1
...,...,...,...,...
18280,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T I unloaded on black celebrities who m...,0
18281,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy s of today grew from the union of sev...,0
18283,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,NATO Russia To Hold Parallel Exercises In Balk...,1


In [18]:
stops = set(stopwords.words("english")) 
df_train['text']=df_train['text'].str.lower()
df_train['text']=df_train['text'].apply(lambda x:' '.join([word for word in x.split() if word.lower() not in (stops)]))
df_train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide even see comey letter jason cha...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might get fired october tension intellig...,1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,videos civilians killed single us airstrike id...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six years prison...,1
...,...,...,...,...
18280,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,rapper unloaded black celebrities met donald t...,0
18281,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,green bay packers lost washington redskins wee...,0
18282,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,macy today grew union several great names amer...,0
18283,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,nato russia hold parallel exercises balkans pr...,1


In [19]:
df_train=df_train.dropna()
df_train=df_train.reset_index()
df_train.drop(['index'],axis=1,inplace=True)
df_train.isna().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [20]:
df_train_fake=df_train[['author']].loc[df_train['label'] == 1]
df_train_real=df_train[['author']].loc[df_train['label'] == 0]
df_train_real=df_train_real['author'].tolist()
df_train_fake=df_train_fake['author'].tolist()
filtered_authors_list=list(set(df_train_real).symmetric_difference(set(df_train_fake)))

In [ ]:
df_train_filtered=pd.DataFrame()
for x in filtered_authors_list:
    df_train_filtered = pd.concat([
    df_train[df_train['author'] == x] for x in filtered_authors_list
    ], ignore_index=True)
df_train_filtered=df_train_filtered.reset_index()
df_train_filtered.drop(['index'],axis=1,inplace=True)
df_train_filtered